In [29]:
import pandas as pd
import gurobipy as gp
import math
import pickle as pkl
from datetime import datetime, timedelta
import copy

In [30]:
product_type = "frozen"
num_time_periods = 365

In [31]:
article_data = pd.read_csv("./data/article.csv")
article_data = article_data[(article_data['TEMPERATURE_ZONE'] == product_type)]
#article_data = article_data.head(465)

In [32]:
# constants definitions
if(product_type=="frozen"):
    warehouse_volume = 20
if(product_type=="chilled"):
    warehouse_volume = 180
if(product_type=="ambient"):
    warehouse_volume = 900
buffer_cost = 25
default_max_order = 10000
max_order_exceed_multiplier = 1.5

In [33]:
def createParameterMatrix(data, columns):
    parameters = []
    for column in columns:
        parameters.append(data[column].to_list())
    parameters = list(map(list, zip(*parameters)))
    return parameters

In [34]:
articles = article_data['ARTICLE_ID'].to_list()

parameters = createParameterMatrix(
    article_data,
    [
        'TEMPERATURE_ZONE',
        'VOLUME_M3_PER_CU',
        'MEAN_SHELF_LIFE',
        'CU_PER_TU',
        'ORDERING_COST_FIXED',
        'ORDERING_COST_PER_TU',
        'CLEARING_COST_PER_CU',
        'MINIMUM_ORDER_QUANTITY_TU',
        'MAXIMUM_ORDER_QUANTITY_TU'
    ]
)
parameters_dict = dict(zip(articles, parameters))

In [35]:
items, category, volume_per_cu, shelf_life, cu_per_tu, ordering_cost_fixed, ordering_cost_per_tu, clearing_cost_per_cu, minimum_order_quantity_tu, maximum_order_quantity_tu = gp.multidict(parameters_dict)

In [36]:
forecast_data = pd.read_csv('./data/sales_'+str(num_time_periods)+'.csv')
forecast_data = forecast_data[forecast_data['ARTICLE_ID'].isin(articles)]

In [37]:
# Create a new dataframe with all dates
all_dates_df = pd.DataFrame({'DATE': pd.date_range(start=min(forecast_data['DATE']), end=max(forecast_data['DATE']), freq='D')}).astype(str)
# Group the original dataframe by item
grouped = forecast_data.groupby('ARTICLE_ID')

# Initialize an empty list to store the new dataframes
new_dfs = []

# Loop over each group
for item, group_df in grouped:
    
    group_df['DATE'] = pd.to_datetime(group_df['DATE']).astype(str)

    # Merge the group dataframe with the all_dates dataframe
    merged_df = pd.merge(all_dates_df, group_df, on='DATE', how='outer')
    merged_df['ARTICLE_ID'] = item
    
    # Fill in missing values
    merged_df['PICKING_QUANTITY_CU'] = merged_df['PICKING_QUANTITY_CU'].fillna(0)
    
    # Sort by date and append to the list
    new_dfs.append(merged_df.sort_values('DATE'))
    
# Concatenate all new dataframes into a single dataframe
forecast_data = pd.concat(new_dfs)
time_periods = forecast_data['DATE'].unique()

time_indexes = [*range(len(time_periods))]
date_to_index = {time_periods[i]:[*range(len(time_periods))][i] for i in time_indexes}
index_to_date = {[*range(len(time_periods))][i]:time_periods[i] for i in time_indexes}

demand = forecast_data.groupby('ARTICLE_ID').apply(lambda x: dict(zip(x['DATE'], x['PICKING_QUANTITY_CU']))).to_dict()
for item in demand.keys():
    demand[item] = dict((date_to_index[key], value) for (key, value) in demand[item].items())

In [38]:
initial_inventory = {}
for item in demand.keys():
    initial_inventory[item] = 0
    for t in time_indexes:
        if(t%7!=6):
            if(sum(list(demand[item].values())[:t+1]) > maximum_order_quantity_tu[item] * cu_per_tu[item] * ((t+1)-((t+1)//7))):
                initial_inventory[item] += sum(list(demand[item].values())[:t+1]) - (maximum_order_quantity_tu[item] * cu_per_tu[item] * ((t+1)-((t+1)//7)))


In [39]:
def exact_solver(demand,a,b):
# model object
    m = gp.Model()

    # decision variables
    # Xit
    orders = m.addVars(items, time_indexes, vtype=gp.GRB.INTEGER, lb=0)
    # Yit
    ordered_boolean = m.addVars(items, time_indexes, vtype=gp.GRB.BINARY, lb=0)
    # Sit
    storage_used = m.addVars(items, time_indexes, vtype=gp.GRB.INTEGER, lb=0)
    # Zt
    buffer_storage_used = m.addVars(time_indexes, vtype=gp.GRB.INTEGER, lb=0)
    # Dit
    clearances = m.addVars(items, time_indexes, vtype=gp.GRB.INTEGER, lb=0)
    # Qit
    item_max_order_exceeded = m.addVars(items, time_indexes, vtype=gp.GRB.INTEGER, lb=0)

    # objective function
    ordering_cost_per_tu_objective = gp.quicksum(ordering_cost_per_tu[item] * orders[item, t] for item in items for t in time_indexes)
    ordering_cost_fixed_objective = gp.quicksum(ordering_cost_fixed[item] * ordered_boolean[item, t] for item in items for t in time_indexes)
    buffer_storage_objective = gp.quicksum(buffer_cost * buffer_storage_used[t] for t in time_indexes)
    clearance_objective = gp.quicksum(clearing_cost_per_cu[item] * clearances[item, t] for item in items for t in time_indexes)
    max_order_objective = gp.quicksum(ordering_cost_per_tu[item] * max_order_exceed_multiplier * item_max_order_exceeded[item, t] for item in items for t in time_indexes)

    m.setObjective(ordering_cost_per_tu_objective + ordering_cost_fixed_objective + buffer_storage_objective + clearance_objective + max_order_objective, sense=gp.GRB.MINIMIZE)

    # constraints
    # demand satisfaction
    for item in demand.keys():
        for t in time_indexes:
            if(t==0):
                m.addConstr(
                    (orders[item, t] * cu_per_tu[item])
                    # +
                    # initial_inventory[item]
                    -
                    storage_used[item, t]
                    -
                    clearances[item, t]
                    >=
                    demand[item][t]
                    
                )
            else:
                m.addConstr(
                    (orders[item, t] * cu_per_tu[item])
                    +
                    storage_used[item, t-1]
                    -
                    storage_used[item, t]
                    -
                    clearances[item, t]
                    >=
                    demand[item][t]
                )

    # inventory volume constraint
    for t in time_indexes:
        if(t==0):
            m.addConstr(
                gp.quicksum(
                    volume_per_cu[item] * 
                    (
                        (cu_per_tu[item] * orders[item, t])
                    )
                    for item in items
                )
                <=
                warehouse_volume
                +
                buffer_storage_used[t]
            )
        else:
            m.addConstr(
                gp.quicksum(
                    volume_per_cu[item] *
                    (
                        storage_used[item,t-1]
                        +
                        (cu_per_tu[item] * orders[item, t])
                    )
                    for item in items
                )
                <=
                warehouse_volume
                +
                buffer_storage_used[t]
            )

    # min/max constraints (linking too)
    for item in demand.keys():
        for t in time_indexes:
            m.addConstr(
                orders[item, t]
                >=
                minimum_order_quantity_tu[item] * ordered_boolean[item, t]
            )
            if((not math.isnan(maximum_order_quantity_tu[item])) and t%7!=6):
                m.addConstr(orders[item, t] <= (maximum_order_quantity_tu[item] * ordered_boolean[item, t]) + item_max_order_exceeded[item, t])

    for item in demand.keys():
        for t in time_indexes:
            life = shelf_life[item]
            if(t >= life):
                m.addConstr(
                    clearances[item, t]
                    >=
                    gp.quicksum(
                        cu_per_tu[item] * orders[item, t1] for t1 in time_indexes[:t-life]
                    )
                    -
                    gp.quicksum(
                        demand[item][t1] for t1 in time_indexes[:t]
                    )
                    -
                    gp.quicksum(
                        clearances[item, t1] for t1 in time_indexes[:t-1]
                    )
                )

    # Making the decision variables null for all week other than one
    for item in demand.keys():
        for t in time_indexes:
            if (t<a or t >b):
                m.addConstr(orders[item,t]==0)
                m.addConstr(ordered_boolean[item,t]==0)


    # sunday constraint
    for item in demand.keys():
        for t in time_indexes:
            if(t%7==6):
                m.addConstr(orders[item, t] == 0)
                m.addConstr(ordered_boolean[item, t] == 0)
                m.addConstr(clearances[item, t] == 0)
                m.addConstr(storage_used[item, t] == storage_used[item, t-1])

    m.setParam('TimeLimit', 200)
    m.optimize()
    output=m.objVal
    return output


In [40]:
#demand

In [41]:
def data_set_modifier(demand_reset,a,b):
    for xyz in demand_reset:
        abc=demand_reset[xyz]
        for d in abc:
             if d < a or d > b:
                 abc[d]=0

    #print(demand_reset)
    output = exact_solver(demand_reset,a,b)
    return output

   
Fin_output=0
num_days=362
num_weeks=num_days//7
week_indices=[i*7 for i in range(num_weeks)]

for week_start in week_indices:
    week_end=week_start+6
    modified_dataset = copy.deepcopy(demand)
    output=data_set_modifier(modified_dataset,week_start,week_end)
    Fin_output += output

Set parameter TimeLimit to value 200
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 719873 rows, 760848 columns and 1581700 nonzeros
Model fingerprint: 0xf2be46da
Variable types: 0 continuous, 760848 integer (152097 binary)
Coefficient statistics:
  Matrix range     [2e-04, 4e+02]
  Objective range  [1e-01, 7e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective 109909.55000
Presolve removed 717453 rows and 756716 columns
Presolve time: 1.47s
Presolved: 2420 rows, 4132 columns, 9366 nonzeros
Found heuristic solution: objective 89892.750000
Variable types: 0 continuous, 4132 integer (576 binary)
Found heuristic solution: objective 85343.250000

Root relaxation: objective 6.983828e+04, 3234 iterations, 0.01 seconds (0.01 work uni

In [42]:
print(Fin_output)
#

3580641.1499938928
